# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-4fa92731bb-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Disney explorative search") 

Consider the following exploratory information need:

> investigate films produced by Disney and compare workers among films.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36479`    | The Lion King      | node |





Also consider

```
wd:Q36479 ?p ?obj .
```

is the BGP to retrieve all **properties of The Lion King**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for Walt Disney Animation Studios

3. Identify the BGP for Walt Disney (person)

4. List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

5. Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

6. Identify the BGP for the film "The Little Mermaid"

7. Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label). 

8. Identify the BGP for Academy Awards

9. For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [14]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   FILTER regex(?name , 'genre')
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
1


Final query for this task

In [10]:
# write your final query
# find the differernt type of film
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q36479 wdt:P136 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2143665'), ('name', "children's film")]
[('o', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama')]
[('o', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film')]
[('o', 'http://www.wikidata.org/entity/Q28968511'), ('name', 'traditionally animated film')]
4


## Task 2
Identify the BGP for Walt Disney Animation Studios

In [24]:
# write your queries
# by filter at first find the sutible object
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   ?s ?p ?o .
   # get the label
   ?o sc:name ?name.
   FILTER REGEX(?name,'Walt Disney Animation Studios')
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1047410'), ('name', 'Walt Disney Animation Studios')]
[('o', 'http://www.wikidata.org/entity/Q56070713'), ('name', 'Walt Disney Animation Studios film')]
[('o', 'http://www.wikidata.org/entity/Q101099318'), ('name', 'Walt Disney Animation Studios film universe')]
[('o', 'http://www.wikidata.org/entity/Q6603742'), ('name', 'list of Walt Disney Animation Studios films')]
[('o', 'http://www.wikidata.org/entity/Q7461830'), ('name', 'Category:Walt Disney Animation Studios films')]
[('o', 'http://www.wikidata.org/entity/Q25088476'), ('name', 'Category:Walt Disney Animation Studios')]
[('o', 'http://www.wikidata.org/entity/Q24424974'), ('name', 'Walt Disney Animation Studios Reveals New Zootopia Concept Art')]
[('o', 'http://www.wikidata.org/entity/Q28382036'), ('name', 'Walt Disney Animation Studios: The Archive Series')]
[('o', 'http://www.wikidata.org/entity/Q15056184'), ('name', 'Template:Walt Disney Animation Studios')]
[('o', 'http://www.w

Final query for this task

In [26]:
# write your final query
# we need specify the products
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p wd:Q1047410 .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P272'), ('name', 'production company')]
1


In [29]:
# write your queries
# finaly we find the task issue
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q36479 wdt:P272 wd:Q1047410;
               wdt:P31 ?o.
   # get the label
   ?o sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q29168811'), ('name', 'animated feature film')]
1


## Task 3
Identify the BGP for Walt Disney (person)

In [11]:
# write your queries
# find the properties of Disney
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    # bind something
    wd:Q1047410 ?p ?o.
    # get the label
    ?p sc:name ?name.
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('name', 'director / manager')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/

Final query for this task

In [17]:
# write your final query
# find BGP of Walt Disney by using related subject from before part
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    # bind something
    wd:Q1047410 wdt:P112 ?o.
    # get the label
    ?o sc:name ?name.
    FILTER regex(?name , 'Walt Disney').
}
LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8704'), ('name', 'Walt Disney')]
1


## Task 4
List all the films created by the Walt Disney Animation Studios while Walt Disney was alive (the result set must be a list of couples films IRI and label).

In [ ]:
# write your queries

Final query for this task

In [ ]:
# write your final query

## Task 5
Find the name of the voice actors who participated in more than 5 films by the Walt Disney Animation Studios (the result set must be a list of triples voice actors IRI, label and #films).

In [22]:
# write your queries
# at first finding the voice actor
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   FILTER regex(?name , 'voice')
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P725'), ('name', 'voice actor')]
1


In [28]:
# write your queries
# link two part together
queryString = """
SELECT DISTINCT ?film_IRI  ?voiceactor_IRI ?voiceactorname 
WHERE {
    ?film_IRI wdt:P725 ?voiceactor_IRI .
    ?voiceactor_IRI sc:name ?voiceactorname
}

LIMIT 30
"""
print("Results")
x=run_query(queryString)

Results
[('film_IRI', 'http://www.wikidata.org/entity/Q20035735'), ('voiceactor_IRI', 'http://www.wikidata.org/entity/Q5372060'), ('voiceactorname', 'Emily Bett Rickards')]
[('film_IRI', 'http://www.wikidata.org/entity/Q51221761'), ('voiceactor_IRI', 'http://www.wikidata.org/entity/Q5372060'), ('voiceactorname', 'Emily Bett Rickards')]
[('film_IRI', 'http://www.wikidata.org/entity/Q3233480'), ('voiceactor_IRI', 'http://www.wikidata.org/entity/Q7670520'), ('voiceactorname', 'TJ Ramini')]
[('film_IRI', 'http://www.wikidata.org/entity/Q7234355'), ('voiceactor_IRI', 'http://www.wikidata.org/entity/Q7670520'), ('voiceactorname', 'TJ Ramini')]
[('film_IRI', 'http://www.wikidata.org/entity/Q104898016'), ('voiceactor_IRI', 'http://www.wikidata.org/entity/Q14917610'), ('voiceactorname', 'Jakub Kohák')]
[('film_IRI', 'http://www.wikidata.org/entity/Q18031874'), ('voiceactor_IRI', 'http://www.wikidata.org/entity/Q3490854'), ('voiceactorname', 'Luděk Sobota')]
[('film_IRI', 'http://www.wikidata.or

In [30]:
# write your queries
# find propertise about the related subject
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [32]:
# write your queries
# from task 2 : BGP of Walt Disney Animation Studios film is Q56070713 
queryString = """
SELECT DISTINCT ?film ?name 
WHERE
{
    ?film wdt:P179 wd:Q56070713 ;
    sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('film', 'http://www.wikidata.org/entity/Q223163'), ('name', "The Emperor's New Groove")]
[('film', 'http://www.wikidata.org/entity/Q134430'), ('name', 'Snow White and the Seven Dwarfs')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana')]
[('film', 'http://www.wikidata.org/entity/Q199839'), ('name', 'The Jungle Book')]
[('film', 'http://www.wikidata.org/entity/Q216929'), ('name', 'Oliver & Company')]
[('film', 'http://www.wikidata.org/entity/Q329316'), ('name', 'The Black Cauldron')]
[('film', 'http://www.wikidata.org/entity/Q537407'), ('name', 'Mulan')]
[('film', 'http://www.wikidata.org/entity/Q270470'), ('name', 'Peter Pan')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('name', 'Beauty and the Beast')]
[('film', 'http://www.wikidata.org/entity/Q215365'), ('name', 'Brother Bear')]
[('film', 'http://www.wikidata.org/entity/Q936194'), ('name', 'Home on the Range')]
[('film', 'http://www.wikidata.org/entity/Q36479'), ('name', 'The Lion King')]
[(

Final query for this task

In [42]:
# write your final query
# by using parallel querry for find voice actor name and number of it's film and group by there
queryString = """
SELECT DISTINCT ?voice_actor ?voice_actor_name (COUNT(?film) AS ?films_num)
WHERE{
    {
        SELECT DISTINCT ?voice_actor ?voice_actor_name ?film 
        WHERE{
            ?film wdt:P725 ?voice_actor .
            ?voice_actor sc:name ?voice_actor_name .
            {
                SELECT DISTINCT ?film 
                WHERE{
                    ?film wdt:P179 wd:Q56070713 .
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film
                WHERE {
                    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
                    ?film wdt:P272 wd:Q1047410 ;
                        # get the type of film
                        wdt:P31 ?instanceof .
                    # we are considering only films that are instance of 'film', 'anthology film', 'animated film', '3D film','feature film', 'live-action/animated film','animated feature film' 
                    # and 'Walt Disney Animation Studios film'
                    FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        }
    }
} GROUP BY ?voice_actor ?voice_actor_name
HAVING (COUNT(?film)>5)
"""
 
print("Results")
x=run_query(queryString)

Results
[('voice_actor', 'http://www.wikidata.org/entity/Q296577'), ('voice_actor_name', 'Frank Welker'), ('films_num', '14')]
[('voice_actor', 'http://www.wikidata.org/entity/Q328456'), ('voice_actor_name', 'Jennifer Darling'), ('films_num', '7')]
[('voice_actor', 'http://www.wikidata.org/entity/Q2451247'), ('voice_actor_name', 'Pat Buttram'), ('films_num', '6')]
[('voice_actor', 'http://www.wikidata.org/entity/Q349350'), ('voice_actor_name', 'Alan Tudyk'), ('films_num', '7')]
[('voice_actor', 'http://www.wikidata.org/entity/Q3603'), ('voice_actor_name', 'Jack Angel'), ('films_num', '7')]
[('voice_actor', 'http://www.wikidata.org/entity/Q24574600'), ('voice_actor_name', 'Terri Douglas'), ('films_num', '6')]
[('voice_actor', 'http://www.wikidata.org/entity/Q2739967'), ('voice_actor_name', 'Jimmy MacDonald'), ('films_num', '7')]
[('voice_actor', 'http://www.wikidata.org/entity/Q310292'), ('voice_actor_name', 'Jim Cummings'), ('films_num', '16')]
[('voice_actor', 'http://www.wikidata.org

In [42]:
# write your final query
queryString = """
SELECT DISTINCT ?voice_actor ?voice_actor_name (COUNT(?film) AS ?films_num)
WHERE{
    {
        SELECT DISTINCT ?voice_actor ?voice_actor_name ?film 
        WHERE{
            ?film wdt:P725 ?voice_actor .
            ?voice_actor sc:name ?voice_actor_name .
            {
                SELECT DISTINCT ?film 
                WHERE{
                    ?film wdt:P179 wd:Q56070713 .
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film
                WHERE {
                    
                    ?film wdt:P272 wd:Q1047410 ;
                          wdt:P31 ?instanceof .
                    FILTER (?instanceof IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        }
    }
} GROUP BY ?voice_actor ?voice_actor_name
HAVING (COUNT(?film)>5)
"""
 
print("Results")
x=run_query(queryString)

Results
[('voice_actor', 'http://www.wikidata.org/entity/Q296577'), ('voice_actor_name', 'Frank Welker'), ('films_num', '14')]
[('voice_actor', 'http://www.wikidata.org/entity/Q328456'), ('voice_actor_name', 'Jennifer Darling'), ('films_num', '7')]
[('voice_actor', 'http://www.wikidata.org/entity/Q2451247'), ('voice_actor_name', 'Pat Buttram'), ('films_num', '6')]
[('voice_actor', 'http://www.wikidata.org/entity/Q349350'), ('voice_actor_name', 'Alan Tudyk'), ('films_num', '7')]
[('voice_actor', 'http://www.wikidata.org/entity/Q3603'), ('voice_actor_name', 'Jack Angel'), ('films_num', '7')]
[('voice_actor', 'http://www.wikidata.org/entity/Q24574600'), ('voice_actor_name', 'Terri Douglas'), ('films_num', '6')]
[('voice_actor', 'http://www.wikidata.org/entity/Q2739967'), ('voice_actor_name', 'Jimmy MacDonald'), ('films_num', '7')]
[('voice_actor', 'http://www.wikidata.org/entity/Q310292'), ('voice_actor_name', 'Jim Cummings'), ('films_num', '16')]
[('voice_actor', 'http://www.wikidata.org

## Task 6
Identify the BGP for the film "The Little Mermaid"

In [85]:
# write your queries
#find the related subject
queryString = """
SELECT DISTINCT ?s  ?name
WHERE {
   # bind something
   ?s ?p wd:Q36479.
   # get the label
   ?s sc:name ?name .  
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1838284'), ('name', 'Shenzi, Banzai and Ed')]
[('s', 'http://www.wikidata.org/entity/Q21772103'), ('name', 'Ed')]
[('s', 'http://www.wikidata.org/entity/Q27044293'), ('name', 'The Lion King')]
[('s', 'http://www.wikidata.org/entity/Q2287748'), ('name', 'Mufasa')]
[('s', 'http://www.wikidata.org/entity/Q134138'), ('name', 'The Lion King')]
[('s', 'http://www.wikidata.org/entity/Q6069146'), ('name', 'Irene Mecchi')]
[('s', 'http://www.wikidata.org/entity/Q203960'), ('name', 'James Earl Jones')]
[('s', 'http://www.wikidata.org/entity/Q191803'), ('name', "The Lion King II: Simba's Pride")]
[('s', 'http://www.wikidata.org/entity/Q451833'), ('name', 'Ernie Sabella')]
[('s', 'http://www.wikidata.org/entity/Q491264'), ('name', 'Nathan Lane')]
[('s', 'http://www.wikidata.org/entity/Q73544451'), ('name', 'The Lion King universe')]
[('s', 'http://www.wikidata.org/entity/Q1646614'), ('name', 'Hakuna Matata')]
[('s', 'http://www.wikidata.org/entity/Q2

Final query for this task

In [95]:
# write your final query
# by using filter easily we can find the excepted result
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q944287 ?p ?o .
   # get the label
   ?o sc:name ?name .
   FILTER regex(?name , 'The Little Mermaid')
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q190135'), ('name', 'The Little Mermaid')]
[('o', 'http://www.wikidata.org/entity/Q6599789'), ('name', 'list of The Little Mermaid episodes')]
2


## Task 7
Find who worked on both "The Little Mermaid" and "The Lion King" (the result set must be a list of couples worker IRI and label).

In [96]:
# write your queries
#at first find the useful propertise
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q944287 ?p ?o .
   # get the label
   ?p sc:name ?name .
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2364'), ('name', 'production code')]
[('p', 'http://www.wikidata.org/prop/direct/P2437'), ('name', 'number of seasons')]
[('p', 'http://www.wikidata.org/prop/direct/P2603'), ('name', 'Kinopoisk film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2

In [98]:
# write your queriesqueryString
#find persons name by using the related propertise
queryString = """
SELECT DISTINCT ?person ?personname
WHERE {
   # bind something
   wd:Q944287 ?p ?person .
   # get the label
   ?person sc:name ?personname.
   FILTER (?p IN (wdt:P161, wdt:P170, wdt:P725))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q3565642'), ('personname', 'Walt Disney Television')]
[('person', 'http://www.wikidata.org/entity/Q3015230'), ('personname', 'Danny Cooksey')]
[('person', 'http://www.wikidata.org/entity/Q1138602'), ('personname', 'Jodi Benson')]
[('person', 'http://www.wikidata.org/entity/Q1264306'), ('personname', 'Pat Carroll')]
[('person', 'http://www.wikidata.org/entity/Q926362'), ('personname', 'Samuel E. Wright')]
[('person', 'http://www.wikidata.org/entity/Q5335683'), ('personname', 'Edan Gross')]
[('person', 'http://www.wikidata.org/entity/Q847124'), ('personname', 'Maurice LaMarche')]
[('person', 'http://www.wikidata.org/entity/Q1356690'), ('personname', 'Kenneth Mars')]
[('person', 'http://www.wikidata.org/entity/Q310292'), ('personname', 'Jim Cummings')]
[('person', 'http://www.wikidata.org/entity/Q304701'), ('personname', 'Bradley Pierce')]
10


In [104]:
# write your queries
# find all involved people

queryString = """
SELECT DISTINCT ?person ?personname (COUNT(?film) AS ?filmcount)
WHERE {
   # bind 'The Lion King' or 'The Little Mermaid' to the persons who worked on it
   ?film ?p ?person .
   # get the label
   ?person sc:name ?personname .
   
   FILTER (?film IN (wd:Q36479,wd:Q944287)) .
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483,wdt:161))
} GROUP BY ?person ?personname
HAVING (COUNT(?film)>1)
LIMIT 200
"""

print("Results")
x=run_query(queryString)


Results
[('person', 'http://www.wikidata.org/entity/Q310292'), ('personname', 'Jim Cummings'), ('filmcount', '2')]
1


In [100]:
# write your queries
queryString = """
SELECT DISTINCT ?person ?personname
WHERE {
   # bind something
   wd:Q190135 ?p ?person .
   # get the label
   ?person sc:name ?personname.
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483))
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q1341627'), ('personname', 'Rod McKuen')]
[('person', 'http://www.wikidata.org/entity/Q2396556'), ('personname', 'Mark Henn')]
[('person', 'http://www.wikidata.org/entity/Q11853913'), ('personname', 'Barry Cook')]
[('person', 'http://www.wikidata.org/entity/Q12320808'), ('personname', 'Jorgen Klubien')]
[('person', 'http://www.wikidata.org/entity/Q28790229'), ('personname', 'Phillip Young')]
[('person', 'http://www.wikidata.org/entity/Q28803503'), ('personname', 'Robert Stanton')]
[('person', 'http://www.wikidata.org/entity/Q28813470'), ('personname', 'Vera Pacheco')]
[('person', 'http://www.wikidata.org/entity/Q3498870'), ('personname', 'Steve Bulen')]
[('person', 'http://www.wikidata.org/entity/Q7375942'), ('personname', 'Ruben A. Aquino')]
[('person', 'http://www.wikidata.org/entity/Q369276'), ('personname', 'Doug Lefler')]
[('person', 'http://www.wikidata.org/entity/Q1497666'), ('personname', 'Geefwee Boedoe')]
[('person', 'http:/

Final query for this task

In [103]:
# write your final query

queryString = """
SELECT DISTINCT ?person ?personname (COUNT(?film) AS ?filmcount)
WHERE {
   # bind something
   ?film ?p ?person .
   # get the label
   ?person sc:name ?personname .
   
   FILTER (?film IN (wd:Q36479,wd:Q190135)) .
   FILTER (?p IN (wdt:P3092, wdt:P1040, wdt:P175, wdt:P162, wdt:P57, wdt:P58, wdt:P86, wdt:P2554, wdt:P1431, wdt:P3174, wdt:P725, wdt:P6942, wdt:P4483))
} GROUP BY ?person ?personname
HAVING (COUNT(?film)>1)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q7381129'), ('personname', 'Russ Edmonds'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q28790229'), ('personname', 'Phillip Young'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5365068'), ('personname', 'Ellen Woodbury'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5228386'), ('personname', 'Dave Bossert'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q28813259'), ('personname', 'Christine Blum'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q2966604'), ('personname', 'Chris Jenkins'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q4772378'), ('personname', 'Anthony DeRosa'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q7363958'), ('personname', 'Ron Husband'), ('filmcount', '2')]
[('person', 'http://www.wikidata.org/entity/Q5238736'), ('personname', 'David Pruiksma'), ('filmcount', '2')]
[('person

## Task 8
Identify the BGP for Academy Awards

In [113]:
# write your queries
# find the nominated propertise 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?o.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [117]:
# write your queries
# find the related object
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q36479 wdt:P1411 ?o .
   # get the label
   ?o sc:name ?name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q488651'), ('name', 'Academy Award for Best Original Score')]
1


In [106]:
# write your queries

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q488651 ?p ?obj .
   
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7084'), ('name', 'related category')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P856'), ('name', 'official website')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
[('p', 'http://www.wikidata.org/prop/direct/

Final query for this task

In [118]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q488651 wdt:P31 ?obj .
   
   # get the label
   ?obj sc:name ?name.
   FILTER regex(?name , 'Academy Awards')
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
1


## Task 9
For each films created by the Walt Disney Animation Studios return the number of nomination for an Academy Awards. Return only films with at least one nomination. (the result set must be triples of films IRI, label and #nomination).

In [119]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q19020 ?p ?o.
   
   # get the label
   ?p sc:name ?name.
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1407'), ('name', 'MusicBrainz series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003

In [4]:
# write your queries
queryString = """
SELECT DISTINCT ?film ?filmname ?nominated_for ?nominated_for_name
WHERE {
    # bind something
    ?film wdt:P272 wd:Q1047410.
    # get the label 
    ?film sc:name ?filmname ;
           wdt:P1411 ?nominated_for .       
    ?nominated_for sc:name ?nominated_for_name.
    FILTER regex(?nominated_for_name, "academy", "i")
}
LIMIT 40
"""

print("Results")
x=run_query(queryString)


Results
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('filmname', 'Moana'), ('nominated_for', 'http://www.wikidata.org/entity/Q106800'), ('nominated_for_name', 'Academy Award for Best Animated Feature')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('filmname', 'Beauty and the Beast'), ('nominated_for', 'http://www.wikidata.org/entity/Q488651'), ('nominated_for_name', 'Academy Award for Best Original Score')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('filmname', 'Beauty and the Beast'), ('nominated_for', 'http://www.wikidata.org/entity/Q102427'), ('nominated_for_name', 'Academy Award for Best Picture')]
[('film', 'http://www.wikidata.org/entity/Q179673'), ('filmname', 'Beauty and the Beast'), ('nominated_for', 'http://www.wikidata.org/entity/Q830079'), ('nominated_for_name', 'Academy Award for Best Sound')]
[('film', 'http://www.wikidata.org/entity/Q379873'), ('filmname', 'Who Framed Roger Rabbit'), ('nominated_for', 'http://www.wikidata.org/entity/Q393686

In [132]:
# write your queries
queryString = """
SELECT DISTINCT ?film ?filmname (COUNT(?nominated_for) AS ?number_nominated)
WHERE {
    # bind something
    ?film wdt:P272 wd:Q1047410.
    # get the label 
    ?film sc:name ?filmname ;
           wdt:P1411 ?nominated_for .
           
    ?nominated_for sc:name ?nominated_for_name.
    FILTER regex(?nominated_for_name, "academy", "i")
}GROUP BY ?film ?filmname
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('filmname', 'Moana'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q2399982'), ('filmname', 'Tea for Two Hundred'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q1633009'), ('filmname', 'The Little Matchgirl'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q3259450'), ('filmname', 'Lorenzo'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q842306'), ('filmname', 'Saludos Amigos'), ('number_nominated', '2')]
[('film', 'http://www.wikidata.org/entity/Q16857388'), ('filmname', 'Feast'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q1440760'), ('filmname', 'Runaway Brain'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q215617'), ('filmname', 'Sleeping Beauty'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q246283'), ('filmname', 'Frozen'), ('number_nominated', '1')

In [122]:
# write your queries


Final query for this task

In [135]:
# write your final query
queryString = """
SELECT DISTINCT ?film ?name (COUNT(?nominated_for) AS ?number_nominated) WHERE{

    ?film wdt:P1411 ?nominated_for .
    ?nominated_for sc:name ?nominated_for_name.
    FILTER regex(?nominated_for_name, "academy", "i")

   {
       
   
        SELECT DISTINCT ?film ?name WHERE {
            {
                SELECT DISTINCT ?film ?name WHERE{
                    ?film wdt:P179 wd:Q56070713 ;
                       sc:name ?name.
                }
            }
            UNION
            {
                # the films having as 'production company' 'Walt Disney Animation Studios'
                SELECT DISTINCT ?film ?name WHERE {
                    # set the 'production company' as 'Walt Disney Animation Studios' to retrieve all its films
                    ?film wdt:P272 wd:Q1047410.
                    # get the label
                    ?film sc:name ?name ;
                          wdt:P31 ?instance_of .
                    FILTER (?instance_of IN (wd:Q11424,wd:Q336144,wd:Q202866,wd:Q229390,wd:Q24869,wd:Q25110269,wd:Q29168811,wd:Q56070713))
                }
            }
        }
        
   }
}GROUP BY ?film ?name
LIMIT 40
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q134430'), ('name', 'Snow White and the Seven Dwarfs'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q190135'), ('name', 'The Little Mermaid'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q18647981'), ('name', 'Moana'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q215365'), ('name', 'Brother Bear'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q4341553'), ('name', 'Pinocchio'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q842306'), ('name', 'Saludos Amigos'), ('number_nominated', '2')]
[('film', 'http://www.wikidata.org/entity/Q36092'), ('name', 'Lilo & Stitch'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q189875'), ('name', 'Alice in Wonderland'), ('number_nominated', '1')]
[('film', 'http://www.wikidata.org/entity/Q40895'), ('name', 'Dumbo'), ('number_nominated', '1')]
[('film', 'http://w